In [3]:
import os
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from file_utils import get_docs_for_plan, pdf_to_text, clean_and_split,sentencewise_translate, temp_get_docs_for_plan
from models import get_embedding_model, get_gemini_llm, get_claude_llm, get_mistral_llm, get_commandR_llm
from RAG import get_answer, get_answer_foreach_doc, diffrent_question_rephrasing, get_top_k_by_count
import pandas as pd
import time
import pandas as pd
from itertools import chain
import torch
import dotenv

dotenv.load_dotenv()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cpu'

# read data

In [4]:
data = pd.read_csv('shpan.csv')
data[["pl_number", 'pl_name']].head()

,pl_number,pl_name
0,101-0253286,תוספת שטחי בניה עיקריים עבור מרפסות מקורות בתנאים
1,101-0209593,רכבת קלה בירושלים - קו ירוק
2,101-0699561,"כביש 396, בין צומת כרם להדסה"
3,101-0721050,"מתחם הוג'יטק-שינוי הבנוי המאושר בתכנית 6394, ג..."
4,תתל/ 108/ סעיף-78-77,מסילת רכבת בירושלים


# - convret and split each plane - from file_utils.py 
# - and do embedding for each plane and save it in db

In [7]:
embedding_model = get_embedding_model()

if os.path.exists("faiss_index"):
    db = FAISS.load_local("faiss_index", embedding_model)
else:
    all_docs = []
    for i, row in data.iterrows():
        files_text = (pdf_to_text(doc) for doc in temp_get_docs_for_plan(row['pl_number']))

        # for each doc, clean and split to chunks
        files_cleand_chunks = [clean_and_split(text, row['pl_number']) for text in files_text]
        flattened_documents = list(chain(*files_cleand_chunks))

        # print(f"Found {len(flattened_documents)} documents for {row['pl_number']}")
        if len(flattened_documents) == 0:
            continue
        all_docs.extend(flattened_documents)
    db = FAISS.from_documents(all_docs, embedding_model)
    db.save_local("faiss_index")

# try our program with gemini that uses one chunk that the retrival return and ask the model one question

In [8]:
if os.path.exists("result_n1_k0_gemini_binary.csv"):
    df_gemini_n1_k0 = pd.read_csv("result_n1_k0_gemini_binary.csv")
else:
    llm = get_gemini_llm(rate_limit=60)
    question = "האם הבניה המתוכננת היא ברחוב אורוגוואי?"
    result = get_answer_foreach_doc(question, db, data, num_docs=1, num_rephrasings=0, model=llm, verbose=False, multiprocess=4)

    # save the result
    df_gemini_n1_k0 = pd.DataFrame(result)
    df_gemini_n1_k0.to_csv('result_n1_k0_gemini_binary.csv', index=False)

In [23]:
df_gemini_n1_k0['pl_name'] = data['pl_name']
df_gemini_n1_k0[df_gemini_n1_k0['answer']=='כן']

# df_gemini_n1_k0['answer'].value_counts()

,pl_number,answer,chunks,pl_name
31,101-0252619,כן,"['- חוק התכנון והבניה, התשכ""ה1965\nהוראות התכנ...",תוספת בניה בקומת מרתף והכשרת עבירת בניה למרפסת...
133,101-0339010,כן,['יעודי קרקע ושימושים 4.\n מגורים ומסחר 4.1\nש...,"הרחבת מבנה מסחרי ברח' אורוגואי 31, קרית היובל,..."
180,101-0345298,כן,"['תנאים למתן היתרי בניה 6.9\n(\'מרתפי חניה, מת...",אור גנים
271,101-0917013,כן,"['- חוק התכנון והבניה, התשכ""ה1965\nהוראות התכנ...",רחוב אורוגוואי 18 ירושלים - תוספת זכויות בניה
372,101-1112010,כן,"['הקיים. הבנייה באבן נסורה, חלקה, ללא סיתות, א...","הרחבת יח""ד קיימות - אורוגוואי 18 ירושלים"
429,101-0845842,כן,"['- חוק התכנון והבניה, התשכ""ה1965\nהוראות התכנ...","תוספות בניה בכל קומות הבניין לשם הרחבות יח""ד, ..."
466,101-1072545,כן,"['- חוק התכנון והבניה, התשכ""ה1965\nהוראות התכנ...","הרחבת יח""ד ברחוב אורגוואי 44, קריית היובל, ירו..."
469,101-1059435,כן,['דרך מוצעת 4.2\nשימושים4.2.1\n - דרך כהגדרתה ...,"הרחבות יח""ד קיימות ברח' אורוגואי 50א' ירושלים"


as we can see the model is not able to answer the question - not all the planes in uruguay he say yes and the one that he say yes is not in uruguay

# try our program with gemini that uses 3 chunkS that the retrival return and ask the model one question

In [10]:
if os.path.exists("result_n3_k0_gemini_binary.csv"):
    df_gemini_n3_k0 = pd.read_csv("result_n3_k0_gemini_binary.csv")
else:
    llm = get_gemini_llm(rate_limit=60)
    question = "האם הבניה המתוכננת היא ברחוב אורוגוואי?"
    result = get_answer_foreach_doc(question, db, data, num_docs=3, num_rephrasings=0, model=llm, verbose=False, multiprocess=4)

    # save the result
    df_gemini_n3_k0 = pd.DataFrame(result)
    df_gemini_n3_k0.to_csv('result_n3_k0_gemini_binary.csv', index=False)

In [11]:
df_gemini_n3_k0[df_gemini_n3_k0['answer']=='כן']

df_gemini_n3_k0['answer'].value_counts()

answer
אין ברשותי מספיק מידע.                                                                             338
No document found                                                                                  168
לא                                                                                                  20
כן                                                                                                   8
אין ברשותי מספיק מידע                                                                                5
אין ברשותי מספיק מידע כדי לענות על השאלה.                                                            4
אין ברשותי מספיק מידע. הקטעים שסופקו אינם מזכירים את רחוב אורוגוואי בהקשר לבנייה המתוכננת.           1
אין ברשותי מספיק מידע כדי לקבוע האם הבנייה המתוכננת היא ברחוב אורוגוואי.                             1
הקטעים שסופקו אינם מכילים מידע על רחוב אורוגוואי, לכן אין ברשותי מספיק מידע כדי לענות על השאלה.      1
אין ברשותי מספיק מידע כדי לקבוע אם הבנייה המתוכננת היא ברחוב אורוג

as we can see the model is not able to answer the question - not all the planes in uruguay he say yes and the one that he say yes is not in uruguay

# try our program with gemini that uses 5 chunkS that the retrival return and ask the model one question

In [12]:
if os.path.exists("result_n5_k0_gemini_binary.csv"):
    df_gemini_n5_k0 = pd.read_csv("result_n5_k0_gemini_binary.csv")
else:
    llm = get_gemini_llm(rate_limit=60)
    question = "האם הבניה המתוכננת היא ברחוב אורוגוואי?"
    result = get_answer_foreach_doc(question, db, data, num_docs=5, num_rephrasings=0, model=llm, verbose=False, multiprocess=4)

    # save the result
    df_gemini_n5_k0 = pd.DataFrame(result)
    df_gemini_n5_k0.to_csv('result_n5_k0_gemini_binary.csv', index=False)

In [13]:

df_gemini_n5_k0[df_gemini_n5_k0['answer']=='כן']

df_gemini_n5_k0['answer'].value_counts()

answer
אין ברשותי מספיק מידע.                                                                                                                          300
No document found                                                                                                                               168
לא                                                                                                                                               28
אין ברשותי מספיק מידע כדי לענות על השאלה.                                                                                                        14
כן                                                                                                                                                8
אין ברשותי מספיק מידע                                                                                                                             6
אין ברשותי מספיק מידע כדי לקבוע אם הבנייה המתוכננת היא ברחוב אורוגוואי.                                  

as we can see the model is not able to answer the question - not all the planes in uruguay he say yes and the one that he say yes is not in uruguay

In [14]:
if os.path.exists("result_n1_k5_gemini_binary.csv"):
    df_gemini_n1_k5 = pd.read_csv("result_n1_k5_gemini_binary.csv")
else:
    llm = get_gemini_llm(rate_limit=60)
    question = "האם הבניה המתוכננת היא ברחוב אורוגוואי?"
    result = get_answer_foreach_doc(question, db, data, num_docs=1, num_rephrasings=5, model=llm, verbose=False, multiprocess=4)

    # save the result
    df_gemini_n1_k5 = pd.DataFrame(result)
    df_gemini_n1_k5.to_csv('result_n1_k5_gemini_binary.csv', index=False)

In [15]:
df_gemini_n1_k5[df_gemini_n1_k5['answer']=='כן']

df_gemini_n1_k5['answer'].value_counts()

answer
אין ברשותי מספיק מידע.    301
No document found         168
אין ברשותי מספיק מידע      52
לא                         18
כן                          8
Name: count, dtype: int64

In [16]:
if os.path.exists("result_n1_k10_gemini_binary.csv"):
    df_gemini_n1_k10 = pd.read_csv("result_n1_k10_gemini_binary.csv")
else:
    llm = get_gemini_llm(rate_limit=60)
    question = "האם הבניה המתוכננת היא ברחוב אורוגוואי?"
    result = get_answer_foreach_doc(question, db, data, num_docs=1, num_rephrasings=10, model=llm, verbose=False, multiprocess=4)

    # save the result
    df_gemini_n1_k10 = pd.DataFrame(result)
    df_gemini_n1_k10.to_csv('result_n1_k10_gemini_binary.csv', index=False)

In [17]:
df_gemini_n1_k10[df_gemini_n1_k10['answer']=='כן']

df_gemini_n1_k10['answer'].value_counts()

answer
אין ברשותי מספיק מידע.    298
No document found         168
אין ברשותי מספיק מידע      54
לא                         20
כן                          7
Name: count, dtype: int64

# try the same with claude

In [18]:
# read file if exists and if not create it
if os.path.exists('result_n1_k5_claude_binary.csv'):
    df = pd.read_csv('result_n1_k5_claude_binary.csv')
else:
    llm = get_claude_llm()
    question = "האם הבניה המתוכננת היא ברחוב אורוגוואי?"
    result = get_answer_foreach_doc(question, db, data, num_docs=1, num_rephrasings=5, model=llm, verbose=False, multiprocess=False)
    # save the result
    df = pd.DataFrame(result)
    df.to_csv('result_n1_k5_claude_binary.csv', index=False)
df

KeyboardInterrupt: 

dont succseed to get the answer that we want(we told him to answer yes or no but he say too much information that we cant summarize)

# try the same with commandR+

In [19]:
if os.path.exists("result_n3_k10_commandR_binary.csv"):
    df = pd.read_csv("result_n3_k10_commandR_binary.csv")
else:
    llm = get_commandR_llm(rate_limit=10)
    question = "האם הבניה המתוכננת היא ברחוב אורוגוואי?"
    result = get_answer_foreach_doc(question, db, data, num_docs=1, num_rephrasings=0, model=llm, verbose=False, multiprocess=False)

    # save the result
    df = pd.DataFrame(result)
    df.to_csv('result_n3_k10_commandR_binary.csv', index=False)

In [20]:
df['answer'].value_counts()
df[(df['answer']=='כן')| (df['answer']=='כן.')]

,pl_number,answer,chunks
31,101-0252619,כן.,"['- חוק התכנון והבניה, התשכ""ה1965\nהוראות התכנ..."
115,101-0387019,כן.,['\'מגורים א 4.1\nתאום תשתיות עם מהנדס הועדה ה...
133,101-0339010,כן.,['יעודי קרקע ושימושים 4.\n מגורים ומסחר 4.1\nש...
158,101-0205674,כן.,['הגדרות בתכנית 1.9\n \nלא רלוונטי\nכל מונח אש...
180,101-0345298,כן.,"['תנאים למתן היתרי בניה 6.9\n(\'מרתפי חניה, מת..."
265,101-0750851,כן.,['יעודי קרקע ושימושים 4.\n \'מגורים ב 4.1\nשימ...
270,101-0596270,כן.,['\'מגורים ד 4.1\n10 . נקודת פליטת האוויר ממער...
271,101-0917013,כן.,"['- חוק התכנון והבניה, התשכ""ה1965\nהוראות התכנ..."
346,101-1010560,כן.,['יעודי קרקע ושימושים 4.\n \'מגורים ב 4.1\nשימ...
372,101-1112010,כן.,"['הקיים. הבנייה באבן נסורה, חלקה, ללא סיתות, א..."


dont succseed to get the answer that we want - he say yes but the plane is not in uruguay

# try the same with misteral 7

In [21]:
if os.path.exists("result_n3_k10_mistral_binary.csv"):
    df = pd.read_csv("result_n3_k10_mistral_binary.csv")
else:
    llm = get_mistral_llm()
    question = "האם הבניה המתוכננת היא ברחוב אורוגוואי?"
    # try to get the answer just on one plane
    result = get_answer_foreach_doc(question, db, data[:1], num_docs=1, num_rephrasings=0, model=llm, verbose=True, multiprocess=False)

    # save the result
    df = pd.DataFrame(result)
    df.to_csv('result_n3_k10_mistral_binary.csv', index=False)
df

ValidationError: 1 validation error for LlamaCpp
__root__
  Could not load Llama model from path: models/mistral-7b-instruct-v0.1.Q4_K_M.gguf. Received error Model path does not exist: models/mistral-7b-instruct-v0.1.Q4_K_M.gguf (type=value_error)

In [ ]:
import re
def parse_answer(ans):
    # get the answer from the string
    reg1 = r'.*התשובה היא:?\s*"?(כן|לא|אין ברשותי מספיק מידע)"?.*'
    reg2 = r'.*התשובה היא "?(כן|לא|אין ברשותי מספיק מידע)"?.*'
    reg3 = r'.*\n.*(כן|לא|אין ברשותי מספיק מידע).*'
    match = re.match(reg1, ans)
    match2 = re.match(reg2, ans)
    match3 = re.match(reg3, ans)
    if match:
        return match.group(1)
    elif match2:
        return match2.group(1)
    # elif match3:
        # return match3.group(1)
    else:
        return ans

df['answer_parsed'] = df['answer'].apply(parse_answer)
df['answer_parsed'].value_counts()

df[df['answer_parsed']=='כן']['answer'].values[0]

In [ ]:
# if os.path.exists("result_n1_k0_gemini_binar
#                   if os.path.exists("result_n3_k0_gemini_binary.csv"):
#     df = pd.read_csv("result_n3_k0_gemini_binary.csv")
# else:
#     llm = get_gemini_llm(rate_limit=True)
#     question = "האם הבניה המתוכננת היא ברחוב אורוגוואי?"
#     result = get_answer_foreach_doc(question, db, data, num_docs=3, num_rephrasings=0, model=llm, verbose=True, multiprocess=False)

#     # save the result
#     df = pd.DataFrame(result)
#     df.to_csv('result_n3_k0_gemini_binary.csv', index=False)y.csv"):
#     df = pd.read_csv("result_n1_k0_gemini_binary.csv")
# else:
#     llm = get_gemini_llm(rate_limit=60)
#     question = "האם הבניה המתוכננת היא ברחוב אורוגוואי?"
#     result = get_answer_foreach_doc(question, db, data, num_docs=1, num_rephrasings=0, model=llm, verbose=True, multiprocess=False)

#     # save the result
#     df = pd.DataFrame(result)
#     df.to_csv('result_n1_k0_gemini_binary.csv', index=False)

In [8]:
# if os.path.exists("result_n3_k0_gemini_binary.csv"):
#     df = pd.read_csv("result_n3_k0_gemini_binary.csv")
# else:
#     llm = get_gemini_llm(rate_limit=True)
#     question = "האם הבניה המתוכננת היא ברחוב אורוגוואי?"
#     result = get_answer_foreach_doc(question, db, data, num_docs=3, num_rephrasings=0, model=llm, verbose=True, multiprocess=False)

#     # save the result
#     df = pd.DataFrame(result)
#     df.to_csv('result_n3_k0_gemini_binary.csv', index=False)

['האם הבניה המתוכננת היא ברחוב אורוגוואי?']
0s document: אין ברשותי מספיק מידע.
1s document: No document found


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised DeadlineExceeded: 504 Deadline Exceeded.


2s document: אין ברשותי מספיק מידע.
3s document: אין ברשותי מספיק מידע.
4s document: No document found
5s document: אין ברשותי מספיק מידע.
6s document: אין ברשותי מספיק מידע.
7s document: No document found
8s document: אין ברשותי מספיק מידע.
9s document: No document found
10s document: No document found
11s document: אין ברשותי מספיק מידע.
12s document: אין ברשותי מספיק מידע.


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised DeadlineExceeded: 504 Deadline Exceeded.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised DeadlineExceeded: 504 Deadline Exceeded.


13s document: אין ברשותי מספיק מידע
14s document: No document found
15s document: No document found
16s document: No document found
17s document: No document found
18s document: אין ברשותי מספיק מידע.
19s document: אין ברשותי מספיק מידע.
20s document: No document found
21s document: No document found
22s document: אין ברשותי מספיק מידע.
23s document: No document found
24s document: אין ברשותי מספיק מידע.
25s document: No document found
26s document: No document found
27s document: אין ברשותי מספיק מידע.
28s document: No document found
29s document: No document found
30s document: No document found


...

541s document: אין ברשותי מספיק מידע.
542s document: אין ברשותי מספיק מידע


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised DeadlineExceeded: 504 Deadline Exceeded.


543s document: אין ברשותי מספיק מידע.
544s document: אין ברשותי מספיק מידע.


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised DeadlineExceeded: 504 Deadline Exceeded.


545s document: אין ברשותי מספיק מידע.
546s document: אין ברשותי מספיק מידע.


In [10]:
df[df['answer']=='כן']

,pl_number,answer,chunks
31,101-0252619,כן,"[- חוק התכנון והבניה, התשכ""ה1965\nהוראות התכני..."
133,101-0339010,כן,[יעודי קרקע ושימושים 4.\n מגורים ומסחר 4.1\nשי...
180,101-0345298,כן,"[תנאים למתן היתרי בניה 6.9\n('מרתפי חניה, מתקנ..."
271,101-0917013,כן,"[- חוק התכנון והבניה, התשכ""ה1965\nהוראות התכני..."
392,101-0977165,כן,[דרך מוצעת 4.2\n \n1 .. תוואי הדרכים רוחבן ו...
424,101-1194000,כן,[יעודי קרקע ושימושים 4.\n מגורים4.1\nשימושים4....
429,101-0845842,כן,"[- חוק התכנון והבניה, התשכ""ה1965\nהוראות התכני..."
466,101-1072545,כן,"[- חוק התכנון והבניה, התשכ""ה1965\nהוראות התכני..."
469,101-1059435,כן,[דרך מוצעת 4.2\nשימושים4.2.1\n - דרך כהגדרתה ב...


In [11]:
if os.path.exists("result_n5_k0_gemini_binary.csv"):
    df = pd.read_csv("result_n5_k0_gemini_binary.csv")
else:
    llm = get_gemini_llm(rate_limit=True)
    question = "האם הבניה המתוכננת היא ברחוב אורוגוואי?"
    result = get_answer_foreach_doc(question, db, data, num_docs=5, num_rephrasings=0, model=llm, verbose=True, multiprocess=False)

    # save the result
    df = pd.DataFrame(result)
    df.to_csv('result_n5_k0_gemini_binary.csv', index=False)

['האם הבניה המתוכננת היא ברחוב אורוגוואי?']
0s document: אין ברשותי מספיק מידע.
1s document: No document found
2s document: אין ברשותי מספיק מידע.
3s document: אין ברשותי מספיק מידע.
4s document: No document found
5s document: אין ברשותי מספיק מידע.
6s document: אין ברשותי מספיק מידע.
7s document: No document found
8s document: אין ברשותי מספיק מידע.
9s document: No document found
10s document: No document found
11s document: אין ברשותי מספיק מידע.


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised DeadlineExceeded: 504 Deadline Exceeded.


12s document: אין ברשותי מספיק מידע.
13s document: אין ברשותי מספיק מידע.
14s document: No document found
15s document: No document found
16s document: No document found
17s document: No document found


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised DeadlineExceeded: 504 Deadline Exceeded.


18s document: אין ברשותי מספיק מידע.
19s document: אין ברשותי מספיק מידע.
20s document: No document found
21s document: No document found
22s document: אין ברשותי מספיק מידע.
23s document: No document found
24s document: אין ברשותי מספיק מידע.
25s document: No document found
26s document: No document found
27s document: אין ברשותי מספיק מידע.
28s document: No document found
29s document: No document found
30s document: No document found
31s document: כן
32s document: No document found
33s document: No document found
34s document: No document found
35s document: אין ברשותי מספיק מידע.
36s document: אין ברשותי מספיק מידע.
37s document: No document found
38s document: No document found
39s document: No document found
40s document: No document found
41s document: No document found
42s document: No document found


...

519s document: לא
520s document: אין ברשותי מספיק מידע
521s document: אין ברשותי מספיק מידע.
522s document: אין ברשותי מספיק מידע כדי לענות על השאלה.
523s document: אין ברשותי מספיק מידע.
524s document: אין ברשותי מספיק מידע.
525s document: לא
526s document: אין ברשותי מספיק מידע.
527s document: אין ברשותי מספיק מידע.
528s document: No document found
529s document: אין ברשותי מספיק מידע.
530s document: אין ברשותי מספיק מידע.


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised DeadlineExceeded: 504 Deadline Exceeded.


531s document: אין ברשותי מספיק מידע.
532s document: אין ברשותי מספיק מידע.
533s document: אין ברשותי מספיק מידע כדי לענות על השאלה.
534s document: אין ברשותי מספיק מידע.
535s document: אין ברשותי מספיק מידע.
536s document: אין ברשותי מספיק מידע.
537s document: אין ברשותי מספיק מידע.
538s document: לא
539s document: אין ברשותי מספיק מידע כדי לענות על השאלה.
540s document: אין ברשותי מספיק מידע.
541s document: אין ברשותי מספיק מידע
542s document: אין ברשותי מספיק מידע.
543s document: אין ברשותי מספיק מידע.
544s document: אין ברשותי מספיק מידע.
545s document: אין ברשותי מספיק מידע.


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised DeadlineExceeded: 504 Deadline Exceeded.


546s document: לא


In [1]:
import pandas as pd
from file_utils import get_docs_for_plan, pdf_to_text, clean_and_split,sentencewise_translate, temp_get_docs_for_plan, clean_text
data = pd.read_csv('shpan.csv')
pl_nums = data['pl_number'].sort_values().unique()
files_text = {pl_num:"\n".join([clean_text(pdf_to_text(doc)) for doc in temp_get_docs_for_plan(pl_num)]) for pl_num in pl_nums}


In [4]:
filtered_docs = [id for id, txt in files_text.items() if len(txt.strip()) > 0]
len(filtered_docs)

379